In [46]:
import numpy as np
import gym
from itertools import product
import matplotlib.pyplot as plt
from tqdm import tqdm

In [47]:
class FourierEstimator:
    def __init__(self, env, n, alpha):
        self.n = n
        self.k = env.observation_space.shape[0]     
        self.n_action = env.action_space.n 
        self.alpha = alpha
        self.w = np.zeros([pow(self.n + 1, self.k), self.n_action])
        self.env = env
        self.alpha_vec = self.setting_alphas()
        
    def setting_alphas(self):
        orders = list(range(self.n + 1))
        c = list(product(orders, repeat = self.k))
        norm_c = np.linalg.norm(c, axis=1)
        norm_c[0] = 1.
        return self.alpha / norm_c
    
    def normalize(self, state):
        norm_state = np.empty(np.shape(state))
        min_state = self.env.observation_space.low    # 바꿀 부분   
        max_state = self.env.observation_space.high    # 바꿀 부분 
        
        for i in range(np.shape(state)[0]):
            state_range = max_state[i] - min_state[i]
            norm_state[i] = float(state[i] - min_state[i]) / state_range
        return norm_state
    
    def get_features(self, state):
        norm_state = self.normalize(state)
        orders = list(range(self.n + 1))
        c = list(product(orders, repeat=self.k))
        return np.cos(np.pi * np.dot(c, norm_state))
    
    def predict_values(self, state):
        return np.dot(self.w.T, self.get_features(state))
    
    def update_weights(self, state, action, target):
        q_values = self.predict_values(state)
        current_q = q_values[action]
        delta = target - current_q
        features = self.get_features(state)
        self.w[:, action] += delta * self.alpha_vec * features

In [42]:
def get_action(s, estimator, epsilon=0):
        values = estimator.predict_values(s)
        a = np.argmax(values)
        if np.random.rand(1) < epsilon:
            a = env.action_space.sample()
        return a

In [49]:
def semi_gradient_sarsa(env, estimator, num_episodes, gamma, epsilon=0):
    step_count = [0] * num_episodes
    for ep in tqdm(range(num_episodes)):
        state = env.reset()
        action = get_action(state, estimator)
        terminated = False
        while not terminated:
            step_count[ep] += 1
            next_state, reward, terminated, _ = env.step(action)
            next_action = get_action(next_state, estimator)
            if terminated:
                target = reward
            else:
                q_values = estimator.predict_values(next_state)
                next_q = q_values[next_action]
                target = reward + gamma * next_q
            estimator.update_weights(state, action, target)
            state = next_state
            action = next_action
                
    return step_count

In [50]:
env = gym.make('MountainCar-v0')

base_alpha = 0.001
fourier_order = 3
estimator = FourierEstimator(env, fourier_order, base_alpha)

num_episodes = 300
gamma = 1
# epsilon = 0
steps = semi_gradient_sarsa(env, estimator, num_episodes, gamma)

plt.plot(steps)
plt.title('Number of steps in an episode')
plt.xlabel('episode')
plt.ylabel('steps')
plt.show()

  0%|          | 0/300 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [51]:
state = env.reset()
h = np.sin(3 * state[0])
height = [h]

while state[0] < 0.5:
    action = get_action(state, estimator)
    state, _, _, _ = env.step(action)
    h = np.sin(3 * state[0])
    height.append(h)
    
plt.figure(1, figsize=(7,4))
plt.plot(height)
plt.xlabel('step')
plt.ylabel('height')
plt.show()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()